Encryption AES, DES, DES3, ARC4, ARC2 in ECB mode

In [3]:
import os
import shutil
from Crypto.Cipher import AES, DES, DES3, ARC4
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes

In [10]:
import os
from Crypto.Cipher import AES, DES, DES3, ARC4, ARC2
from Crypto.Util.Padding import pad
from Crypto.Random import get_random_bytes
import math
import shutil

def encrypt_ecb(algorithm, key, plaintext):
    if algorithm == 'AES':
        cipher = AES.new(key, AES.MODE_ECB)
        return cipher.encrypt(pad(plaintext, AES.block_size))
    elif algorithm == 'DES':
        cipher = DES.new(key, DES.MODE_ECB)
        return cipher.encrypt(pad(plaintext, DES.block_size))
    elif algorithm == 'DES3':
        cipher = DES3.new(key, DES.MODE_ECB)
        return cipher.encrypt(pad(plaintext, DES3.block_size))
    elif algorithm == 'ARC4':
        cipher = ARC4.new(key)
        return cipher.encrypt(plaintext)
    elif algorithm == 'ARC2':
        cipher = ARC2.new(key, ARC2.MODE_ECB)
        return cipher.encrypt(pad(plaintext, ARC2.block_size))

def encrypt_files_by_format_ecb(base_dir, save_dir):
    formats = ['bmp', 'csv', 'html', 'pyth', 'txt', 'wav']
    algorithms = ['AES', 'DES', 'DES3', 'ARC4', 'NONE']

    keys = {
        'AES': get_random_bytes(16),
        'DES': get_random_bytes(8),
        'DES3': DES3.adjust_key_parity(get_random_bytes(24)),
        'ARC4': get_random_bytes(16),
    }

    for folder in formats:
        input_dir = os.path.join(base_dir, folder)
        if not os.path.exists(input_dir):
            continue

        file_list = sorted([f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))])
        total_files = len(file_list)
        files_per_algo = math.ceil(total_files / len(algorithms))

        os.makedirs(save_dir, exist_ok=True)

        for i, algo in enumerate(algorithms):
            start_idx = i * files_per_algo
            end_idx = min((i + 1) * files_per_algo, total_files)
            selected_files = file_list[start_idx:end_idx]

            output_dir = os.path.join(save_dir, f"{folder}_ecb", algo)
            os.makedirs(output_dir, exist_ok=True)

            for file_name in selected_files:
                input_path = os.path.join(input_dir, file_name)
                output_path = os.path.join(output_dir, file_name + ('' if algo == 'NONE' else '.enc'))

                with open(input_path, 'rb') as f:
                    plain_text = f.read()

                if algo == 'NONE':
                    shutil.copy2(input_path, output_path)
                else:
                    cipher_text = encrypt_ecb(algo, keys[algo], plain_text)
                    with open(output_path, 'wb') as f_encrypted:
                        f_encrypted.write(cipher_text)

    print("All files encrypted.")

Encryption

In [4]:
encrypt_files_by_format_ecb('files/1000_files', 'encrypted_ecb/encrypted_ecb_1000/encrypted_ecb_1000_1key')

All files encrypted.


In [5]:
encrypt_files_by_format_ecb('files/1800_files', 'encrypted_ecb/encrypted_ecb_1800/encrypted_ecb_1800_1key')


All files encrypted.


In [6]:
encrypt_files_by_format_ecb('3000_files', 'encrypted_ecb/encrypted_ecb_3000/encrypted_ecb_3000_1key')


All files encrypted.


Encryption using 3 or 6 keys per algo

In [11]:
import math

def generate_keys(keys_per_algorithm):
    keys = {}
    for algo in ['AES', 'DES', 'DES3', 'ARC4']:
        key_list = []
        for _ in range(keys_per_algorithm):
            if algo == 'AES':
                key_list.append(get_random_bytes(16))
            elif algo == 'DES':
                key_list.append(get_random_bytes(8))
            elif algo == 'DES3':
                key_list.append(DES3.adjust_key_parity(get_random_bytes(24)))
            elif algo == 'ARC4':
                key_list.append(get_random_bytes(16))
        keys[algo] = key_list
    return keys

def encrypt_files_by_format_ecb(base_dir, save_dir, keys_per_algorithm=3):
    formats = ['bmp', 'csv', 'html', 'pyth', 'txt', 'wav']
    algorithms = ['AES', 'DES', 'DES3', 'ARC4', 'NONE']

    keys = generate_keys(keys_per_algorithm)

    for folder in formats:
        input_dir = os.path.join(base_dir, folder)
        if not os.path.exists(input_dir):
            continue

        file_list = sorted([f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))])
        total_files = len(file_list)
        files_per_algo = math.ceil(total_files / len(algorithms))

        os.makedirs(save_dir, exist_ok=True)

        for i, algo in enumerate(algorithms):
            start_idx = i * files_per_algo
            end_idx = min((i + 1) * files_per_algo, total_files)
            selected_files = file_list[start_idx:end_idx]

            output_dir = os.path.join(save_dir, f"{folder}_ecb", f"{algo}")
            os.makedirs(output_dir, exist_ok=True)

            for j, file_name in enumerate(selected_files):
                input_path = os.path.join(input_dir, file_name)
                output_path = os.path.join(output_dir, file_name + ('' if algo == 'NONE' else '.enc'))

                with open(input_path, 'rb') as f:
                    plain_text = f.read()

                if algo == 'NONE':
                    shutil.copy2(input_path, output_path)
                else:
                    key_index = j % keys_per_algorithm
                    key = keys[algo][key_index]
                    cipher_text = encrypt_ecb(algo, key, plain_text)
                    with open(output_path, 'wb') as f_encrypted:
                        f_encrypted.write(cipher_text)

    print(f"All files encrypted using {keys_per_algorithm} keys per algorithm.")


In [12]:
encrypt_files_by_format_ecb('files/1000_files', 'encrypted_ecb/encrypted_ecb_1000/encrypted_ecb_1000_3keys', 3)


All files encrypted using 3 keys per algorithm.


In [22]:
encrypt_files_by_format_ecb('files/1000_files', 'encrypted_ecb/encrypted_ecb_1000/encrypted_ecb_1000_6keys', 6)


All files encrypted using 6 keys per algorithm.


In [14]:
encrypt_files_by_format_ecb('files/1800_files', 'encrypted_ecb/encrypted_ecb_1800/encrypted_ecb_1800_3keys', 3)

All files encrypted using 3 keys per algorithm.


In [21]:
encrypt_files_by_format_ecb('files/1800_files', 'encrypted_ecb/encrypted_ecb_1800/encrypted_ecb_1800_6keys', 6)

All files encrypted using 6 keys per algorithm.


In [18]:
encrypt_files_by_format_ecb('files/3000_files', 'encrypted_ecb/encrypted_ecb_3000/encrypted_ecb_3000_3keys', 3)

All files encrypted using 3 keys per algorithm.


In [19]:
encrypt_files_by_format_ecb('files/3000_files', 'encrypted_ecb/encrypted_ecb_3000/encrypted_ecb_3000_6keys', 6)


All files encrypted using 6 keys per algorithm.
